In [16]:
# =========================================================
# PLANT DISEASE CLASSIFICATION - COMPLETE PIPELINE
# TRAINING + EVALUATION + INFERENCE
# =========================================================
import os, torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from datetime import datetime
from PIL import Image


In [23]:
# =========================================================
# CONFIGURATION
# =========================================================
BASE_DIR = r"C:\raw_work_minorproject\disease_detection\Classification\disease_dataset1"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR   = os.path.join(BASE_DIR, "valid")
TEST_DIR  = os.path.join(BASE_DIR, "test")
RESULTS_DIR = "evaluation_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "Resnet_50.pth"
EPOCHS = 20
BATCH_SIZE = 16
LR = 1e-4

In [18]:
# =========================================================
# DATA TRANSFORMS
# =========================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [19]:
# =========================================================
# DATASETS & LOADERS
# =========================================================
train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_ds   = datasets.ImageFolder(VAL_DIR, transform=test_transform)
test_ds  = datasets.ImageFolder(TEST_DIR, transform=test_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

num_classes = len(train_ds.classes)
class_names = train_ds.classes
print(f"Detected {num_classes} plant disease classes.")

Detected 38 plant disease classes.


In [20]:
# =========================================================
# MODEL SETUP (ResNet-50)
# =========================================================
from torchvision.models import resnet50, ResNet50_Weights

# Load pretrained model
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Freeze feature extractor
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer for your number of classes
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=LR)


In [24]:
# =========================================================
# TRAINING LOOP
# =========================================================
best_val_acc = 0.0
train_history = {"train_loss": [], "val_loss": [], "val_acc": []}

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    model.train()
    train_loss = 0.0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, correct = 0.0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    train_history["train_loss"].append(train_loss)
    train_history["val_loss"].append(val_loss)
    train_history["val_acc"].append(val_acc)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), MODEL_PATH)
        print("✅ Best model saved!")


Epoch 1/20
Train Loss: 1.6750 | Val Loss: 0.8634 | Val Acc: 89.14%
✅ Best model saved!

Epoch 2/20
Train Loss: 0.6392 | Val Loss: 0.4812 | Val Acc: 92.62%
✅ Best model saved!

Epoch 3/20
Train Loss: 0.4205 | Val Loss: 0.3551 | Val Acc: 94.04%
✅ Best model saved!

Epoch 4/20
Train Loss: 0.3239 | Val Loss: 0.2595 | Val Acc: 95.10%
✅ Best model saved!

Epoch 5/20
Train Loss: 0.2713 | Val Loss: 0.2405 | Val Acc: 95.35%
✅ Best model saved!

Epoch 6/20
Train Loss: 0.2372 | Val Loss: 0.1986 | Val Acc: 96.04%
✅ Best model saved!

Epoch 7/20
Train Loss: 0.2129 | Val Loss: 0.1782 | Val Acc: 96.19%
✅ Best model saved!

Epoch 8/20
Train Loss: 0.1975 | Val Loss: 0.1616 | Val Acc: 96.48%
✅ Best model saved!

Epoch 9/20
Train Loss: 0.1819 | Val Loss: 0.1607 | Val Acc: 96.37%

Epoch 10/20
Train Loss: 0.1684 | Val Loss: 0.1350 | Val Acc: 96.65%
✅ Best model saved!

Epoch 11/20
Train Loss: 0.1596 | Val Loss: 0.1307 | Val Acc: 96.89%
✅ Best model saved!

Epoch 12/20
Train Loss: 0.1523 | Val Loss: 0.1339

In [ ]:
# =========================================================
# EVALUATION ON TEST SET
# =========================================================
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
print("Total classes (train):", len(class_names))
print("Classes found in test predictions:", len(set(all_labels)))
print("Missing classes:", set(range(len(class_names))) - set(all_labels))